In [2]:
import os
import numpy as np
import pandas as pd
import random

In [329]:
class AssetOverlap():
    def __init__(self):
        self.readFunds()
        self.readSP500()

        
    def readFunds(self):
        fundDir = os.listdir("../input/funds")
        funds = pd.DataFrame(columns=["Ticker", "ID"])
        for name in fundDir:
            fund = pd.read_csv("../input/funds/"+name, index_col=0)
            name = name.split(".csv")[0]
            
            fund = fund.rename(columns={fund.columns[-1]:"Ticker", "ID.WEIGHTS":name})
            fund = fund[["ID","Ticker",name]]
            fund = fund[(fund.Ticker.str[:4]!="#N/A") & (fund[name]>0)] #no puts so neg percent and #N/A Unclassified: Unable to parse request at ...  and # N/A Invalid Security
            fund = fund.dropna(axis=0, how='all')
            
            fund[name] = fund[name]/(fund[name].sum())*100 # some have over 100 percent
            
            fund.Ticker = fund.Ticker.fillna("NULL TICKER"+fund.ID)
            
            fund = fund.groupby(["Ticker","ID"]).agg({name:"sum"})# you have to groupby id and ticker or else it may merge multipletimes
            
            funds = funds.merge(fund, how='outer', on=["Ticker","ID"])
        
        
        params = {fundName:"sum" for fundName in self.getFundNames(funds)}
        params.update({"ID":lambda IDS: IDS.iloc[np.argmax([np.all([not char.isdigit() for char in ID]) for ID in IDS])] }) #first id that is all characters or first id
        funds = funds.groupby("Ticker").agg(params)
        funds = funds.reset_index()

        #print(funds.Ticker.duplicated().sum())
        
        funds = funds.fillna(0)
        self.funds = funds

    def getFundMatrix(self):
        return self.funds.loc[:,~self.funds.columns.isin(["ID","Ticker"])]
    
    def readSP500(self):
        self.sp500 = pd.read_csv("../input/s&p500.csv")
        self.sp500 = self.sp500.iloc[:,:3]
        self.sp500 = self.sp500.rename(columns={"Symbol":"Ticker", "Weight":"S&P500"})
        
    def getFundNames(self,funds):
        return [x for x in funds.columns if x not in ["ID", "Ticker"]]
    
    def makePortfolio(self,weights):
        #self.weights = self.weights.loc[self.getFundNames(self.funds)]
        columns = ["Ticker","ID"] + list(weights.index)
        self.funds = self.funds[columns]

        Portfolio = self.getFundMatrix().dot(weights)
        Portfolio = Portfolio.rename({Portfolio.columns[0]:"Portfolio"},axis=1)
        Portfolio[["Ticker","ID"]] = self.funds[["Ticker","ID"]]
        
        Portfolio = pd.merge(Portfolio, self.sp500, how='outer', on="Ticker")
        Portfolio.ID = Portfolio.ID.fillna(Portfolio.Company)
        Portfolio = Portfolio.drop("Company", axis=1)
        return Portfolio
    
    def makeAddStocksPortfolio(self,Portfolio):
        dif = Portfolio["S&P500"]-Portfolio["Portfolio"] # determine which sp500 assets have more than in the portfolio
        self.extra = Portfolio
        self.extra["Dif"] = dif
        self.extra = self.extra[self.extra.Dif>0][["Ticker","ID","Dif"]]
        self.extra = self.extra.rename(columns={"Dif":"Portfolio"})
        AddStocksPortfolio = pd.concat([Portfolio[["Ticker","ID","Portfolio"]],self.extra],axis=0)
        return AddStocksPortfolio
    
    def simpleOverlap(self,weights):
        Portfolio = self.makePortfolio(weights)
        Portfolio[["Portfolio","S&P500"]] = Portfolio[["Portfolio","S&P500"]].fillna(0)
        overlap = np.minimum(np.array(Portfolio["Portfolio"]),np.array(Portfolio["S&P500"]))
        overlap = overlap.sum()
        total = Portfolio.Portfolio.sum()
        return overlap/total*100
    
    def addStocksOverlap(self,weights):
        Portfolio = self.makePortfolio(weights)
        AddStocksPortfolio = self.makeAddStocksPortfolio(Portfolio)
        return 100/AddStocksPortfolio.Portfolio.sum()*100
        
    def evaluateAssetOverlap(self,weights):
        #self.weights = pd.DataFrame(np.ones((49,1))*-1000,index=self.getFundNames(self.funds))
        return self.simpleOverlap(weights)
        
        
        
        

In [156]:
class Discount:
    def __init__(self):
        self.readDiscount()
    def readDiscount(self):
        self.discount = pd.read_csv("../input/Discounts.csv")
        self.discount[["Discount", "52W Discount","Effective","Distribution"]] = self.discount.apply(lambda x: [x["Discount"].strip("%"),x["52W Discount"].strip("%"), x["Effective"].strip("%"), x["Distribution"].strip("%")], axis=1, result_type='expand')
        self.discount = self.discount.replace("--",0)
        self.discount.iloc[:,1:] = self.discount.iloc[:,1:].astype("float")
        
        longterm = self.discount["52W Discount"]-self.discount["Discount"] #max: a pos 52 week is pos, a neg discount is more pos
        longterm = longterm/np.max(longterm) *100# between -inf and 1
        self.discount["longterm"] = longterm
        current = -self.discount["Discount"] # max: a neg discount is pos
        current = current/np.max(-self.discount["Discount"]) * 100 #between -inf and 1
        self.discount["value"] = np.sum([longterm,2*current],axis=0)/3
        self.discount = self.discount.set_index("Ticker")
    
    def getWeightedDiscount(self,weights):
        self.discount = self.discount.loc[weights.index]
        weights = np.array(weights).reshape(1,len(weights))
        values = np.array(self.discount.Discount).reshape(len(self.discount),1)
        return np.matmul(weights,values)[0][0]
    
    def evaluateDiscount(self,weights):
        self.discount = self.discount.loc[weights.index]
        weights = np.array(weights).reshape(1,len(weights))
        values = np.array(self.discount.value).reshape(len(self.discount),1)
        
        return np.dot(weights,values)[0][0]
        
          

In [456]:
Discount().discount

,Effective,Distribution,Discount,52W Discount,longterm,value
Ticker,,,,,,
ADX,0.00,20.55,-13.27,-13.74,-4.304029,36.468780
AEF,7.90,2.64,-12.89,-14.09,-10.989011,33.155048
AGD,6.94,7.51,-12.70,-13.11,-3.754579,35.023824
AIO,2.95,5.05,-6.89,-10.25,-30.769231,9.423681
AOD,0.09,7.84,-13.81,-13.28,4.853480,41.063699
ASA,0.00,0.09,-15.96,-15.45,4.670330,47.143752
AWP,7.68,9.28,-12.22,-13.72,-13.736264,30.325558
BCX,1.20,6.63,-14.22,-15.29,-9.798535,37.350788
BDJ,0.27,7.19,-9.53,-9.34,1.739927,27.800770


In [483]:
class TabuSet:
    def inTabuSet(self,option):
        fund1, fund2 = option
        tabuList = [adjustment.getFunds() for adjustment in self.TabuSet[:self.numTabu]]
        return [fund1,fund2] in tabuList or [fund2,fund1] in tabuList

In [484]:
class Adjustment():
    def __init__(self,fund1,fund2,value,weights):
        self.fund1 = fund1
        self.fund2 = fund2
        self.value = value
        self.weights = weights
    def getValue(self):
        return self.value
    def getWeights(self):
        return self.weights
    def getFunds(self):
        return [self.fund1, self.fund2]

In [487]:
class runModel(AssetOverlap, Discount,TabuSet):
    def __init__(self):
        self.readFunds()
        self.readSP500()
        self.readDiscount()
        
        self.numOptions = 1
        self.numTabu = 0
        
        N = len(self.getFundNames(self.funds))
        weights = np.random.uniform(0,1.0,(N,1))
        weights = np.divide(weights,np.sum(weights))

        self.weights = pd.DataFrame(weights,columns=["weights"],index=self.getFundNames(self.funds))
        self.TabuSet = []
        #self.findInitialBest()
    
    def Evaluate(self,weights):
        asset = .7*(self.evaluateAssetOverlap(weights))
        discount = .3*self.evaluateDiscount(weights)
        return asset+discount
    
    def changeWeights(self,fund1,fund2,weights,change):
        change = min(weights.loc[fund2][0],change)
        weights.loc[fund1,"weights"] += change
        weights.loc[fund2,"weights"] -= change
        return weights
    
    def EvaluateChange(self,fund1,fund2,weights,change):
        weights = self.changeWeights(fund1,fund2,weights,change)
        return self.Evaluate(weights)
    
    def findIdealChange(self,fund1,fund2,weights,change):
        control = self.Evaluate(weights)
        
        while self.EvaluateChange(fund1,fund2,weights,change)>control and weights.loc[fund2][0]!=0:
            weights = self.changeWeights()
          
             
                
        return control, weights
    
    def makeAdjustment(self,fund1,fund2,weights,change):
        control = self.Evaluate(weights)
        up = self.EvaluateChange(fund1,fund2,weights,change)
        down = self.EvaluateChange(fund2,fund1,weights,change)
        
        if down>control and down>up:
            fund1, fund2 = fund2, fund1
        elif control>=up:
            return control,weights
        
        weights = self.changeWeights(fund1,fund2,weights,change)
        value, weights = self.findIdealChange(fund1,fund2,weights,change)
        
        return value, weights
    
    def randomFunds(self):
        fund2 = random.choice(self.weights.index)
        fund1 = random.choice(self.weights.index)
        return fund1, fund2
    
    def getOption(self,fund1,fund2):
        change = .1
        value, weights = self.makeAdjustment(fund1, fund2, self.weights, .1)
        return Adjustment(fund1=fund1,fund2=fund2,value=value,weights=weights)
    
            
    def runIteration(self):
        options = [self.randomFunds() for x in range(self.numOptions)]

        options = list(filter(lambda option: not self.inTabuSet(option),options))
        
        options = [self.getOption(fund1,fund2) for fund1,fund2 in options]

        bestIndx = np.argmax([option.getValue() for option in options])
        option = options[bestIndx]
        self.weights = option.getWeights()
        self.TabuSet.append(option)
    
    def printEvaluation(self):
        print("Simple Overlap %.2f" % self.simpleOverlap(self.weights))
        print("Add Stocks Overlap %.2f" % self.addStocksOverlap(self.weights))
        print("Weighted Discount %.2f" % self.getWeightedDiscount(self.weights))
        
    def runModel(self):
        self.printEvaluation()
        
        for x in range(1000):
            self.runIteration()
            print("\n\n")
            self.printEvaluation()

In [488]:
obj = runModel()
obj.runModel()

Simple Overlap 34.93
Add Stocks Overlap 63.60
Weighted Discount -8.92



Simple Overlap 35.13
Add Stocks Overlap 63.67
Weighted Discount -8.90



Simple Overlap 35.18
Add Stocks Overlap 63.70
Weighted Discount -8.94



Simple Overlap 35.25
Add Stocks Overlap 63.72
Weighted Discount -8.95



Simple Overlap 36.54
Add Stocks Overlap 64.25
Weighted Discount -8.50



Simple Overlap 36.54
Add Stocks Overlap 64.25
Weighted Discount -8.51



Simple Overlap 36.54
Add Stocks Overlap 64.25
Weighted Discount -8.98



Simple Overlap 38.03
Add Stocks Overlap 64.87
Weighted Discount -8.51



Simple Overlap 38.03
Add Stocks Overlap 64.87
Weighted Discount -8.51



Simple Overlap 37.50
Add Stocks Overlap 64.65
Weighted Discount -9.49



Simple Overlap 38.54
Add Stocks Overlap 65.09
Weighted Discount -9.33



Simple Overlap 38.44
Add Stocks Overlap 65.05
Weighted Discount -9.34



Simple Overlap 38.44
Add Stocks Overlap 65.05
Weighted Discount -9.34



Simple Overlap 38.60
Add Stocks Overlap 65.12
Weigh




Simple Overlap 31.98
Add Stocks Overlap 62.43
Weighted Discount -12.62



Simple Overlap 31.98
Add Stocks Overlap 62.43
Weighted Discount -12.62



Simple Overlap 32.05
Add Stocks Overlap 62.45
Weighted Discount -12.62



Simple Overlap 32.05
Add Stocks Overlap 62.45
Weighted Discount -12.65



Simple Overlap 32.05
Add Stocks Overlap 62.45
Weighted Discount -12.65



Simple Overlap 34.29
Add Stocks Overlap 63.34
Weighted Discount -12.45



Simple Overlap 34.29
Add Stocks Overlap 63.34
Weighted Discount -12.45



Simple Overlap 34.76
Add Stocks Overlap 63.53
Weighted Discount -12.31



Simple Overlap 34.30
Add Stocks Overlap 63.34
Weighted Discount -12.47



Simple Overlap 34.30
Add Stocks Overlap 63.34
Weighted Discount -12.47



Simple Overlap 31.75
Add Stocks Overlap 62.33
Weighted Discount -12.17



Simple Overlap 31.75
Add Stocks Overlap 62.33
Weighted Discount -12.17



Simple Overlap 31.75
Add Stocks Overlap 62.33
Weighted Discount -12.17



Simple Overlap 33.42
Add Stocks Ove




Simple Overlap 26.38
Add Stocks Overlap 60.32
Weighted Discount -10.61



Simple Overlap 30.12
Add Stocks Overlap 61.71
Weighted Discount -11.66



Simple Overlap 26.04
Add Stocks Overlap 60.19
Weighted Discount -11.38



Simple Overlap 26.04
Add Stocks Overlap 60.19
Weighted Discount -11.38



Simple Overlap 26.04
Add Stocks Overlap 60.19
Weighted Discount -11.38



Simple Overlap 26.04
Add Stocks Overlap 60.19
Weighted Discount -11.38



Simple Overlap 25.33
Add Stocks Overlap 59.94
Weighted Discount -12.63



Simple Overlap 25.33
Add Stocks Overlap 59.94
Weighted Discount -12.63



Simple Overlap 25.33
Add Stocks Overlap 59.94
Weighted Discount -12.63



Simple Overlap 25.33
Add Stocks Overlap 59.94
Weighted Discount -12.63



Simple Overlap 29.86
Add Stocks Overlap 61.61
Weighted Discount -11.14



Simple Overlap 29.86
Add Stocks Overlap 61.61
Weighted Discount -11.14



Simple Overlap 29.86
Add Stocks Overlap 61.61
Weighted Discount -11.14



Simple Overlap 29.86
Add Stocks Ove




Simple Overlap 23.41
Add Stocks Overlap 59.25
Weighted Discount -13.12



Simple Overlap 23.41
Add Stocks Overlap 59.25
Weighted Discount -13.12



Simple Overlap 23.35
Add Stocks Overlap 59.23
Weighted Discount -12.96



Simple Overlap 23.35
Add Stocks Overlap 59.23
Weighted Discount -12.96



Simple Overlap 23.35
Add Stocks Overlap 59.23
Weighted Discount -12.96



Simple Overlap 23.35
Add Stocks Overlap 59.23
Weighted Discount -12.96



Simple Overlap 23.35
Add Stocks Overlap 59.23
Weighted Discount -12.96



Simple Overlap 23.35
Add Stocks Overlap 59.23
Weighted Discount -12.96



Simple Overlap 23.35
Add Stocks Overlap 59.23
Weighted Discount -12.96



Simple Overlap 23.35
Add Stocks Overlap 59.23
Weighted Discount -12.96



Simple Overlap 27.08
Add Stocks Overlap 60.57
Weighted Discount -10.84



Simple Overlap 27.08
Add Stocks Overlap 60.57
Weighted Discount -10.84



Simple Overlap 27.08
Add Stocks Overlap 60.57
Weighted Discount -10.84



Simple Overlap 27.08
Add Stocks Ove




Simple Overlap 48.39
Add Stocks Overlap 69.55
Weighted Discount -11.27



Simple Overlap 48.39
Add Stocks Overlap 69.55
Weighted Discount -11.27



Simple Overlap 48.39
Add Stocks Overlap 69.55
Weighted Discount -11.27



Simple Overlap 45.45
Add Stocks Overlap 68.16
Weighted Discount -13.25



Simple Overlap 45.45
Add Stocks Overlap 68.16
Weighted Discount -13.25



Simple Overlap 45.45
Add Stocks Overlap 68.16
Weighted Discount -13.25



Simple Overlap 45.45
Add Stocks Overlap 68.16
Weighted Discount -13.25



Simple Overlap 45.45
Add Stocks Overlap 68.16
Weighted Discount -13.25



Simple Overlap 44.31
Add Stocks Overlap 67.63
Weighted Discount -14.34



Simple Overlap 44.31
Add Stocks Overlap 67.63
Weighted Discount -14.34



Simple Overlap 44.31
Add Stocks Overlap 67.63
Weighted Discount -14.34



Simple Overlap 42.72
Add Stocks Overlap 66.91
Weighted Discount -15.30



Simple Overlap 42.72
Add Stocks Overlap 66.91
Weighted Discount -15.30



Simple Overlap 42.72
Add Stocks Ove




Simple Overlap 47.14
Add Stocks Overlap 68.95
Weighted Discount -13.79



Simple Overlap 47.14
Add Stocks Overlap 68.95
Weighted Discount -13.79



Simple Overlap 47.14
Add Stocks Overlap 68.95
Weighted Discount -13.79



Simple Overlap 47.14
Add Stocks Overlap 68.95
Weighted Discount -13.79



Simple Overlap 47.14
Add Stocks Overlap 68.95
Weighted Discount -13.79



Simple Overlap 47.14
Add Stocks Overlap 68.95
Weighted Discount -13.79



Simple Overlap 47.14
Add Stocks Overlap 68.95
Weighted Discount -13.79



Simple Overlap 44.43
Add Stocks Overlap 67.68
Weighted Discount -13.76



Simple Overlap 44.43
Add Stocks Overlap 67.68
Weighted Discount -13.76



Simple Overlap 44.43
Add Stocks Overlap 67.68
Weighted Discount -13.76



Simple Overlap 41.25
Add Stocks Overlap 66.26
Weighted Discount -13.95



Simple Overlap 47.84
Add Stocks Overlap 69.29
Weighted Discount -12.48



Simple Overlap 49.05
Add Stocks Overlap 69.87
Weighted Discount -12.76



Simple Overlap 49.05
Add Stocks Ove

Simple Overlap 49.10
Add Stocks Overlap 69.89
Weighted Discount -9.31



Simple Overlap 48.88
Add Stocks Overlap 69.78
Weighted Discount -9.35



Simple Overlap 48.88
Add Stocks Overlap 69.78
Weighted Discount -9.35



Simple Overlap 48.88
Add Stocks Overlap 69.78
Weighted Discount -9.35



Simple Overlap 48.88
Add Stocks Overlap 69.78
Weighted Discount -9.35



Simple Overlap 48.88
Add Stocks Overlap 69.78
Weighted Discount -9.35



Simple Overlap 48.88
Add Stocks Overlap 69.78
Weighted Discount -9.35



Simple Overlap 47.31
Add Stocks Overlap 69.03
Weighted Discount -9.75



Simple Overlap 47.31
Add Stocks Overlap 69.03
Weighted Discount -9.75



Simple Overlap 47.31
Add Stocks Overlap 69.03
Weighted Discount -9.75



Simple Overlap 47.31
Add Stocks Overlap 69.03
Weighted Discount -9.75



Simple Overlap 47.31
Add Stocks Overlap 69.03
Weighted Discount -9.75



Simple Overlap 47.31
Add Stocks Overlap 69.03
Weighted Discount -9.75



Simple Overlap 47.31
Add Stocks Overlap 69.03
Weigh




Simple Overlap 27.40
Add Stocks Overlap 60.69
Weighted Discount -11.78



Simple Overlap 27.40
Add Stocks Overlap 60.69
Weighted Discount -11.78



Simple Overlap 27.40
Add Stocks Overlap 60.69
Weighted Discount -11.78



Simple Overlap 27.40
Add Stocks Overlap 60.69
Weighted Discount -11.78



Simple Overlap 27.40
Add Stocks Overlap 60.69
Weighted Discount -11.78



Simple Overlap 27.40
Add Stocks Overlap 60.69
Weighted Discount -11.78



Simple Overlap 27.40
Add Stocks Overlap 60.69
Weighted Discount -11.78



Simple Overlap 27.40
Add Stocks Overlap 60.69
Weighted Discount -11.78



Simple Overlap 27.40
Add Stocks Overlap 60.69
Weighted Discount -11.78



Simple Overlap 27.40
Add Stocks Overlap 60.69
Weighted Discount -11.78



Simple Overlap 27.40
Add Stocks Overlap 60.69
Weighted Discount -11.37



Simple Overlap 27.40
Add Stocks Overlap 60.69
Weighted Discount -11.37



Simple Overlap 27.40
Add Stocks Overlap 60.69
Weighted Discount -11.37



Simple Overlap 27.40
Add Stocks Ove

Simple Overlap 35.15
Add Stocks Overlap 63.69
Weighted Discount -14.90



Simple Overlap 35.15
Add Stocks Overlap 63.69
Weighted Discount -14.90



Simple Overlap 35.31
Add Stocks Overlap 63.75
Weighted Discount -15.03



Simple Overlap 35.31
Add Stocks Overlap 63.75
Weighted Discount -15.03



Simple Overlap 35.31
Add Stocks Overlap 63.75
Weighted Discount -15.03



Simple Overlap 35.31
Add Stocks Overlap 63.75
Weighted Discount -15.03



Simple Overlap 35.31
Add Stocks Overlap 63.75
Weighted Discount -15.03



Simple Overlap 28.50
Add Stocks Overlap 61.10
Weighted Discount -15.90



Simple Overlap 28.50
Add Stocks Overlap 61.10
Weighted Discount -15.90



Simple Overlap 28.50
Add Stocks Overlap 61.10
Weighted Discount -15.90



Simple Overlap 35.30
Add Stocks Overlap 63.75
Weighted Discount -14.83



Simple Overlap 35.30
Add Stocks Overlap 63.75
Weighted Discount -14.83



Simple Overlap 38.49
Add Stocks Overlap 65.07
Weighted Discount -14.05



Simple Overlap 38.49
Add Stocks Overla

In [476]:
obj.weights

,weights
STK,0.000000e+00
ADX,1.440363e-01
CHN,0.000000e+00
MIE,0.000000e+00
SZC,0.000000e+00
CII,1.647561e-01
GRF,3.000000e-01
BCX,0.000000e+00
CRF,0.000000e+00
CEN,0.000000e+00


In [352]:
languages = ['Java', 'Python', 'JavaScript']
versions = [14, 3, 6]

result = dict(zip(languages, versions))
result.values()

dict_values([14, 3, 6])

In [341]:
obj = runModel()
obj.trial()

35.66692317114405
0.008299854271928309 0.0002510616573965859
35.67296045982042
35.67296045982042
0.008550915929324894 0.0


In [372]:
def hey(a,b):
    print(a,b)
    
    
hey((1,2))

TypeError: hey() missing 1 required positional argument: 'b'

In [480]:
null = funds[funds["Ticker"].str[:4]=="NULL"]
notnull = funds[funds["Ticker"].str[:4]!="NULL"].groupby("ID")["Ticker"].first()
null.merge(notnull, how="left", left_on="ID", right_index=True)
#no overlap between null and having ticker



#tickers where there is more than one id
k = funds.groupby(["Ticker"])["ID"].count().to_frame()
t = funds.groupby(["Ticker","ID"])["BUI"].count()
t[t.index.get_level_values("Ticker").isin(k[k.ID>1].index)].reset_index().groupby("Ticker")["ID"].apply(hey)

,weights
STK,0.029072
ADX,0.223149
CHN,0.000000
MIE,0.000000
SZC,0.000000
CII,0.049814
GRF,0.017187
BCX,0.000000
CRF,0.153040
CEN,0.000000


In [403]:
Discount().discount

,Effective,Distribution,Discount,52W Discount,value
Ticker,,,,,
ADX,0.00,20.55,-13.27,-13.74,-26.275577
AEF,7.90,2.64,-12.89,-14.09,-22.119033
AGD,6.94,7.51,-12.70,-13.11,-25.329223
AIO,2.95,5.05,-6.89,-10.25,0.624547
AOD,0.09,7.84,-13.81,-13.28,-32.011144
ASA,0.00,0.09,-15.96,-15.45,-36.525396
AWP,7.68,9.28,-12.22,-13.72,-19.310103
BCX,1.20,6.63,-14.22,-15.29,-25.563458
BDJ,0.27,7.19,-9.53,-9.34,-21.285559


In [199]:
FundAssets().addStocksOverlap()

61.39535179302859

In [198]:
FundAssets().simpleOverlap()

33.12524075359048

In [89]:
a = FundAssets().Portfolio.groupby("Company").agg({"S&P500":'count'})

STK
0
ADX
1
CHN
1
MIE
3
SZC
14


In [115]:
dict = {x:"sum" for x in range(2)}
dict.update({"h":'h'})

In [2]:
dict(zip([1,2],["sum","sum"]))

{1: 'sum', 2: 'sum'}

In [1]:
class runModel(AssetOverlap, Discount):
    def __init__(self):
        self.readFunds()
        self.readSP500()
        self.readDiscount()
        
        N = len(self.getFundNames(self.funds))
        weights = np.random.uniform(0,1.0,(N,1))
        weights = np.divide(weights,np.sum(weights))

        self.weights = pd.DataFrame(weights,columns=["weights"],index=self.getFundNames(self.funds))
        
        self.n = .1
        self.run_model()
        
    def Evaluate(self,weights):
        asset = .7*(self.evaluateAssetOverlap(weights))
        discount = .3*-self.evaluateDiscount(weights)
        return asset+discount
        
    def randomFunds(self):
        fund2 = random.choice(self.weights[self.weights["weights"]!=0].index)
        fund1 = random.choice(self.weights.index)
        return fund1, fund2
    
    def run_model(self):
        fund1, fund2 = self.randomFunds()

        for x in range(100):
            newWeights = self.weights.copy()
            
            change = .01
            if newWeights.loc[fund2][0] < change:
                change = newWeights.loc[fund2][0]
            if change == 0 or fund1==fund2: 
                fund1, fund2 = self.randomFunds()
                continue
                
            newWeights.loc[fund1] = [newWeights.loc[fund1]+change]
            newWeights.loc[fund2] = [newWeights.loc[fund2]-change]

            if self.Evaluate(newWeights) > self.Evaluate(self.weights):
                self.weights=newWeights.copy()
            else:
                fund1, fund2 = self.randomFunds()
        
        print(self.evaluateAssetOverlap(self.weights))
        print(self.addStocksOverlap(self.weights))
        print(self.getWeightedDiscount(self.weights))


    

{'a': 1, 'b': 2, 'c': 3}


In [ ]:
foobar = 3.141592
print(f'My number is {foobar:.2f} - look at the nice rounding!')

My number is 3.14 - look at the nice rounding!

In [ ]:
# class runModel(AssetOverlap, Discount):
#     def __init__(self):
#         self.readFunds()
#         self.readSP500()
#         self.readDiscount()
        
#         N = len(self.getFundNames(self.funds))
#         weights = np.random.uniform(0,1.0,(N,1))
#         weights = np.divide(weights,np.sum(weights))

#         self.weights = pd.DataFrame(weights,index=self.getFundNames(self.funds))
        
#         self.n = .1
#         self.run_gradient_descent()
    
#     def run_gradient_descent(self):
#         for x in range(100):
#             gradient = -self.n *self.gradient()
#             self.weights = self.weights + gradient 
#             print("k",self.evaluateAssetOverlap())
#             print(self.evaluateDiscount())
            
#     def gradient(self):
#         asset = .5*(100-self.evaluateAssetOverlap())
#         discount = .5*self.evaluateDiscount()
#         print(asset,discount)
#         return asset+discount